# Neuroidal Model Simulation v1.3
#### Patrick Perrine

Code for:

    Perrine, P. R. (2023). Neural Tabula Rasa: Foundations for Realistic Memories and Learning. Master's thesis. California Polytechnic State University, San Luis Obispo.

The algorithms implemented here are adapted from or inspired by:

    Valiant, L. G. (2005). Memorization and Association on a Realistic Neural Model. Neural Computation, 17, 527–555. https://doi.org/10.1162/0899766053019890

Many thanks to Mugizi Rwebangira and Chandradeep Chowdhury for their constant support.

## Introduction
We offer the following simulation as an accurate, centralized interpretation of the Neuroidal model, as specifically described by Valiant (2005). We place an emphasis on simulating the model to study the behavior of the one-step, shared memory representation for use in unsupervised memorization via the JOIN algorithm. We offer a general implementation framework that should easily extend to the other variants of the model. We offer a new algorithmic primitive *quick JOIN* (simplified as *QJOIN*) which allows for a significantly faster instance of JOIN to be executed for a centralized representation of the Neuroidal model.

We omit an implementation of the LINK procedure, as we do not measure this model based on its ability to *associate* items of information. We are primarily concerned with how much information can be stored using JOIN before an intolerable amount of interference occurs between new and pre-existing memories. Another critical difference is that our algorithms do not function in a distributed manner, leading to some complexity issues that we mitigate with optimization procedures.

The use of the "mode" structure for storing neuron/synapse parameters are an effort to remain aesthetically similar the original formulations in Valiant (2005), and the earlier work *Circuits of the Mind*. Given our centralized strategy here, there is evidence to suggest that this mode format may be later disregarded.

We choose the Python library *graph-tool* to serve as the basis for storing and performing operations on our random graph model. We opt for this particular library due to its reported performance, code readability, and implementations for visualizing very large, dense graphs (which we leave for future work). We also synonymously refer to nodes as 'neurons', edges as 'synapses', and the graph as 'the model.'

### Install the *graph-tool* library (https://graph-tool.skewed.de/)
The following two code blocks are subject to change based on the library's current hosting location, so referring to the current information from its website may be required.

If one needs to run this code without Jupyter: Ignore the following two blocks and please refer to the library's current documentation for proper installation instructions for the desired setup. Also, be sure to remove the "%%capture" markers in any subsequent code blocks.

In [1]:
%%capture
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [2]:
# Run this block if you are on Google Colab, otherwise skip
%%capture
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

### Import other Required Libraries and Seed Random Number Generator

In [3]:
%%capture
import itertools
import numpy as np
from numpy.random import *
import graph_tool.all as gt

In [4]:
global rng
rng = np.random.default_rng(seed=42)

### Define the Neuroidal Model's Empirical Properties
The following configuration is for simulating a shared memory representation model with one-step mechanisms, with the intention to adhere to results given in the tables of Valiant (2005). The first six parameters are as described in the referenced paper. The $r_{approx}$ is for generating initial memories in a fixed size to be used in JOIN, and requires prior determination as pursuant to the relations defined in the paper. The most important aspect about choosing a value of $r_{approx}$ is ensuring that JOIN will create new memories of approximately the same size.

We also implicitly declare all empirical properties as global variables for convenience.

In [5]:
n = 100000
d = 512
t = 1
k = 32
k_adj = 2.0
k_m = k * k_adj

r_approx = 5420

#### Calculate Edge Existence Probability $p$
In the analysis of Valiant (2005) where $n \ge 10^5$, we have $p = \frac{d}{n}$.

However for lower values of $n$, we find the more general $p = \frac{d}{n-1}$ to be appropriate.

In [6]:
if n >= 10^5:
    p = d / n
else:
    p = d / (n - 1)

## Generate the Graph Model
Here we instantiate a graph model equivalent to the Erdős-Rényi $G=(n,p)$ structure.

### Initialize an Empty Graph

In [7]:
global g
g = gt.Graph(directed=True)

### Populate our Graph with $n$ Neurons

In [8]:
%%capture
g.add_vertex(n)

### Populate our Graph with Random Synapses

In [9]:
def add_gnp_edges(): # Naive implementation
    all_edges = itertools.permutations(range(n), 2)
    for e in all_edges:
        if rng.random() < p:
            g.add_edge(*e)

The following function provides a speed-optimized (yet less readable and more memory intensive) implementation of adding a list of random edges. This version is more reminiscent of the $G=(n,M)$ model, but here we determine the number of edges $M$ using Bernoulli trials.

In [10]:
def add_fast_gnp_edges():
    num_edges = rng.binomial(n*(n-1)/2, p)
    sources = rng.integers(0, n, num_edges*2)
    targets = rng.integers(0, n, num_edges*2)
    mask = sources != targets # removes self-loops
    g.add_edge_list(np.column_stack((sources[mask], targets[mask])))

In [11]:
#add_gnp_edges() # Space-Efficient
add_fast_gnp_edges() # Time-Efficient

### Initialize the Mode of all Neurons and Synapses
The following two blocks imbue the global properties, or the "mode" $s$, of each node and edge in the graph.

For each neuron $i$, its mode $s_i$ contains the following properties:
* $T_i$ specifies the threshold gate value of a given node, commonly set to $1$, or as high as $7$.
* $q_i$ stores the state of a given node, initialized as $1$ and set to $2$ if a node is found to be a candidate for memory $C$ during JOIN.
  * During *two-step* procedures, this state can also be set to $3$.
  * This can be interpreted as the state $q_i$ belonging to a set of finite states $Q = \{1,2,3\}$.
      * Such states can be representative of how a given neuron is considered "allocated" in memory or otherwise.
* $f_i$ is a number indicating whether a given neuron is firing ($1$) or not firing ($0$) at the current time step.

For efficiency purposes, we omit the storing of the incoming weight sum, $w_{i}$, within in the mode, as it will be calculated in the weight summation functions to be described later.

In [12]:
mode_T = g.new_vp("int")
mode_q = g.new_vp("int")
mode_f = g.new_vp("int")

mode_T.a = t
mode_q.a = 1
mode_f.a = 0

For each synapse $(j,i)$, its mode $s_{ji}$ contains the following:
* $w_{ji}$ specifies the weight of the edge from node $j$ to node $i$.
  * For *disjoint* representations, this would be initialized to be $\frac{t}{k}$.
  * However, for *shared* representations, we initialize the weight to be $\frac{t}{k_m}$.
* $qq_{ji}$ stores the state of a given edge, set to $1$ and is not updated for one-step procedures.
  * For two-step operations, we would have $qq_{ji} \in Q = \{1,2\}$.

In [13]:
mode_w = g.new_ep("double")
mode_qq = g.new_ep("int")

mode_w.a = t / k_m
mode_qq.a = 1

## Vicinal Algorithms for the JOIN Algorithm

### Weight Summation Functions
The following two blocks both calculate and return
$$ w_i = \sum_{(j,i)\in E}{} f_{j} \cdot w_{ji} $$
given a neuron $i$.

In [14]:
def sum_weights(s_i): # Naive implementation
    w_i = 0
    for s_ji in g.iter_in_edges(s_i):
        if mode_f[s_ji[0]] == 1:
            w_i += mode_w[s_ji]
    return w_i

This method below is equivalent to the function above, but leverages speed optimizations offered by graph-tool and NumPy to negate the costly, explicit loop.

In [15]:
def fast_sum_weights(s_i):
    W = g.get_in_edges(s_i, [mode_w])[:,2]
    F = np.array(g.get_in_neighbors(s_i, [mode_f])[:,1], dtype=bool)
    return W[F].sum()

### Neuron and Synapse Mode Updates

$\delta\left(s_{i}, w_{i}\right) = s_{i}'$.

In [16]:
def _delta(s_i, w_i):
    if w_i > mode_T[s_i]:
        mode_f[s_i] = 1
        mode_q[s_i] = 2

$\lambda\left(s_{i}, w_{i}, s_{ji}, f_{j}\right) = s_{ji}'$.

In [17]:
def _lambda(s_i, w_i, s_ji, f_j):
    if f_j == 1:
        mode_qq[s_ji] = 2

### Overall Graph Update
Note that traversal of each node to calculate the sum of its incoming weights results in one of our most costly procedures in this simulation. We attribute this partly because the Neuroidal model was conceived as a *distributed system* with neurons and synapses acting as their own *agents*. Here they are expressed simply as objects with their attributes being manipulated by our algorithms. Hence, we are performing operations sequentially for each node in the graph, and therefore acknowledge that this overall process is not quite exact to the original formulation of the model.

We allow a user to utilize one-step mechanisms using a Boolean function argument. We also allow the skipping of updating the edges of the graph using this argument, as edge updates are not needed in the one-step case. After all updates, we reset the states and firing modes of each neuron/synapse, indicating that the *time step* has been completed.

#### Shortcut for Determining Memory $C$
For convenience, we check for eligible $C$ nodes during the weight sums for each neuron of the graph.

In [18]:
def update_graph(one_step=True):
    C = []
    for s_i in g.iter_vertices():
        w_i = fast_sum_weights(s_i)
        _delta(s_i, w_i)
        if mode_q[s_i] == 2:
            C.append(s_i)
            if one_step:
                mode_f[s_i] = 0
        if not one_step:
            for s_ji in g.iter_in_edges(s_i):
                f_j = mode_f[s_ji[0]]
                _lambda(s_i, w_i, s_ji, f_j)
    return C

## The JOIN Algorithm
This implements the *one-step* variant of JOIN for *shared* representations as
defined in Valiant (2005), explained in further detail here:

1. Fire all neurons in given memories $A$ and $B$ "simultaneously."
2. Update the graph to determine any threshold transitions.
3. Create memory $C$ from neurons that transitioned.

In [19]:
def JOIN_one_step_shared(A, B):
    for i in A + B:
        mode_f[i] = 1
    C = update_graph(one_step=True)
    mode_f.a = 0
    mode_q.a = 1
    return C

## QJOIN: An Optimized JOIN for Centralized Representations
The following function makes a couple of additional assumptions about the Neuroidal model and leverages them into a significantly faster algorithm for a centralized implementation:

1. Set the weights of all synapses in the model to be $0$.
2. Set the weights of the outgoing synapses of $A \cup B$ to be $\frac{t}{k_m}$.
3. Create memory $C$ from neurons that transitioned.

In [20]:
def quick_JOIN(A, B):
    mode_w.a = 0
    for i in A + B:
        mode_w.a[g.get_out_edges(i, [g.edge_index])[:,2]] = t / k_m
    return g.get_vertices()[g.get_in_degrees(g.get_vertices(),eweight=mode_w)>t]

QJOIN does not assume bipartiteness between memories, nor does it require the use of the mode structure aside from storing synapse weights. It does assume that the *synapse weights from the previous time step can be erased*. It also assumes that all neurons have the same threshold value of $t$ at the given time step.

It is currently unknown if this version of JOIN remains biologically plausible.

## Interference Check Function
Here we check every memory that is not $A$ or $B$ (referred to as $D$ memories) to calculate how many interfering nodes exist between each given $D$ memory and $C$. If more than $50\%$ of a $D$ memory is found to interfere with $C$, then an instance of interference has been found. We would then increment the sum of occurrences by $2$, as we consider interference to be bidirectional to both incidental memories.

In [21]:
def interference_check(S, A_i, B_i, C):
    sum = 0
    for D_i in range(len(S)):
        if D_i != A_i and D_i != B_i:
            D = S[D_i]
            if len(set(C) & set(D)) > (len(D) / 2):
                sum += 2
    return sum

## Our Simulation

### Print Functions for Simulation Updates
These three blocks are simply for providing feedback for when: A batch of ongoing JOIN operations completes, the model's capacity is reached, or all memory connectives have been memorized.

The interference metric reports are mostly useful for when $n < 10^5$, where the overall system is generally much less stable. Hence, we omit the frequent reporting of it when $n$ is above that value.

In [22]:
def print_join_update(S_length, L, H, H_if, total_if, m_len, m_total):
    print("Current Total Memories:", S_length)
    print("Batch Average Memory Size:", int(m_len/H))
    print("Running Average Memory Size:", int(m_total/(S_length-L)),"\n\n")
    if n < 10^5:
        print("Batch Interference Rate:", round(H_if/H, 6))
        print("Running Average Int. Rate:", round(total_if/S_length, 6))

In [23]:
def print_halt_msg(S_length, L, F, total_if, m_total):
    r_obs = int(m_total/(S_length-L))
    r_error = round(((r_approx - r_obs) / r_obs) * 100, 2)
    print("-- End of Simulation (Halted) --\n")
    print("Given: n=", n, "d=", d, "k=", k, "k_adj=", k_adj, "\n r_approx=",
           r_approx, "START_MEM=", L)
    print("we halted Memory Formation at", F*100, "% Total Interference.\n")
    print("Empirical Memory Size:", int(m_total/(S_length-L)))
    print("Approximation Error of r:", r_error, "%")
    print("Total Average Interference Rate:", round(total_if/S_length, 6))
    print("Capacity:", L, "Initial Memories +" , S_length-L,"JOIN Memories.")

In [24]:
def print_memorized_msg(S_length, L, F, m_total):
    r_obs = int(m_total/(S_length-L))
    r_error = round(((r_approx - r_obs) / r_obs) * 100, 2)
    print("-- End of Simulation (Completed) --\n")
    print("Given: n=", n, "d=", d, "k=", k, "k_adj=", k_adj, "\n r_approx=",
           r_approx, "START_MEM=", L, ":\n")
    print("We memorized all combinations of",L,"memories","\n","with less than",
           F*100, "% interference.\n")
    print("Empirical Memory Size:", int(m_total/(S_length-L)))
    print("Approximation Error of r:", r_error, "%")
    print("Contains:", L, "Initial Memories +" ,S_length-L,"JOIN Memories.")

### Call JOIN until Capacity is reached or all connectives are Memorized

#### Simulation Variables
* The $L$ parameter specifies how many initial memories to grant the model before calling JOIN.
  * Each initial memory will be of fixed size $r_{approx}$ and will consist of random neurons chosen with replacement.
* $S$ is our "memory bank," in which we will initially append $L\choose{2}$ combinations of initial memories, and continue to add memories and check interference as a result of JOIN.
* The $F$ parameter determines the level of interference tolerance to be checked at each iteration of JOIN.
* (Optional) The $H$ parameter determines how many memories to JOIN in one batch of simulation.
  * This is used to observe the model perform at an adjustable level of granularity.
* All other variables are primarily used for update reporting.

Our simulation will attempt to JOIN all possible $L\choose{2}$ pairs of initial memories until the model's capacity is reached. If $L\choose{2}$ is found to be less than our capacity, then the simulation will terminate and have found the model to have memorized all possible connectives.

In [25]:
def JOIN_one_step_shared_simulation(L, F, H=1, fast=True, verbose=True):
    m = 0
    H_if = 0
    m_len = 0
    m_total = 0
    total_if = 0
    print("-- Start of Simulation --\n")
    init_pairs = itertools.combinations(range(L), 2)
    S = [rng.choice(np.arange(0,n-1), size=r_approx)
         for _ in range(L)]
    for A_i,B_i in init_pairs:
        A = list(S[A_i])
        B = list(S[B_i])
        if fast:
            C = quick_JOIN(A, B)
        else:
            C = JOIN_one_step_shared(A, B)
        C_if = interference_check(S, A_i, B_i, C)
        m += 1
        S.append(C)
        m_len += len(C)
        m_total += len(C)
        if m % H == 0:
            if verbose:
                print_join_update(len(S), L, H, H_if,
                                  total_if, m_len, m_total)
            H_if = 0
            m_len = 0
        if C_if > 0:
            H_if += C_if
            total_if += C_if
            if total_if/len(S) > F:
              print_halt_msg(len(S), L, F, total_if, m_total)
              return S
    print_memorized_msg(len(S), L, F, m_total)
    return S

### Generate our Memory Bank and Calculate the Empirical Memory Size
Given the default parameters, we observe that there is no notable amount of interference (less than $10^{-6}$) for the given amount of memories disjuncted. We are then able to verify the precision in our choice of $r_{approx}$ empirically. We also later observe that the observed capacity of the shared model is clearly greater than the analytical capacity for a disjoint representation.

In [26]:
memory_bank = JOIN_one_step_shared_simulation(L=10,
                                              F=.00001,
                                              fast=True,
                                              H=10, verbose=True)

-- Start of Simulation --

Current Total Memories: 20
Batch Average Memory Size: 5459
Running Average Memory Size: 5459 


Current Total Memories: 30
Batch Average Memory Size: 5373
Running Average Memory Size: 5416 


Current Total Memories: 40
Batch Average Memory Size: 5422
Running Average Memory Size: 5418 


Current Total Memories: 50
Batch Average Memory Size: 5392
Running Average Memory Size: 5411 


-- End of Simulation (Completed) --

Given: n= 100000 d= 512 k= 32 k_adj= 2.0 
 r_approx= 5420 START_MEM= 10 :

We memorized all combinations of 10 memories 
 with less than 0.001 % interference.

Empirical Memory Size: 5425
Approximation Error of r: -0.09 %
Contains: 10 Initial Memories + 45 JOIN Memories.


#### Compare Results with the Capacity for the *Disjoint* Representation Model
Capacity is measured here by using a result from Table 1 of Valiant (2005).

In [27]:
print("Analytical capacity of disjoint representation:", int(100000 / 5420))

Analytical capacity of disjoint representation: 18
